In [1]:
#Import Libraries and print their versions
#%run /content/drive/MyDrive/Bayesian\ path\ analysis/imports.py
%run imports.py

Python version:     3.11.12
pymc version:       5.21.2
numpy version:      2.0.2
pandas version:     2.2.2
arviz version:      0.21.0
matplotlib version: 3.10.0


In [2]:
#%run /content/drive/MyDrive/Bayesian\ path\ analysis/model_B02.py
%run model_B02.py

In [3]:
def run_and_summarize(data, model_func, label="b02", output_dir="../results"):
    """
    Runs the model, summarizes results, saves outputs: idata, summary, metrics, plots.
    Works for B01 or B02 depending on the model_func provided.
    """
    os.makedirs(output_dir, exist_ok=True)

    model, idata, waic, loo, bic = model_func(data["predictor"], data["outcome"])

    # Save InferenceData
    idata_path = os.path.join(output_dir, f"idata_{label}.nc")
    idata.to_netcdf(idata_path)

    # Determine var_names based on model
    try:
        # Check if variables like "effect" and "sd_fluctuation" exist (i.e. B02)
        var_names = ["intercept", "effect", "sd_fluctuation", "level_effects", "p"]
        _ = idata.posterior["effect"]
    except KeyError:
        # B01 fallback
        var_names = ["intercept", "level_effects", "p"]

    # Save Summary
    summary = az.summary(idata, var_names=var_names)
    summary_path = os.path.join(output_dir, f"survey_summary_{label}.csv")
    summary.to_csv(summary_path)

    # Save WAIC, LOO, BIC as a markdown file
    metrics_path = os.path.join(output_dir, f"survey_model_metrics_{label}.md")
    with open(metrics_path, "w") as f:
        f.write(f"## Model {label.upper()} Metrics\n\n")
        f.write("### WAIC\n```\n")
        f.write(waic.to_string() + "\n```\n")
        f.write("### LOO\n```\n")
        f.write(loo.to_string() + "\n```\n")
        f.write(f"### BIC\n```\n\nBIC = {bic:.2f}\n```")

    # Save Plots
    trace_plot_path = os.path.join(output_dir, f"survey_trace_{label}.png")
    az.plot_trace(idata, var_names=[vn for vn in var_names if vn != "p"])
    plt.tight_layout()
    plt.savefig(trace_plot_path, dpi=300)
    plt.close()

    posterior_plot_path = os.path.join(output_dir, f"survey_posterior_p_{label}.png")
    az.plot_posterior(idata, var_names=["p"])
    plt.tight_layout()
    plt.savefig(posterior_plot_path, dpi=300)
    plt.close()

    ppc_plot_path = os.path.join(output_dir, f"survey_ppc_{label}.png")
    az.plot_ppc(idata)
    plt.tight_layout()
    plt.savefig(ppc_plot_path, dpi=300)
    plt.close()

    print(f"✅ Saved all outputs for model {label.upper()} in '{output_dir}' folder.")

    return idata, waic, loo, bic


In [5]:
# Data Generation
#%run /content/drive/MyDrive/Bayesian\ path\ analysis/test_data.py
%run test_data.py

In [6]:
#Generate test data
test_data = generate_test_data(random_seed=42)

# Run and save results for B02
print("\nResults for B02 Model:")
idata_b02, waic_b02, loo_b02, bic_b02 = run_and_summarize(test_data, ordinal_predictor_binary_outcome_model, label="b02")


Results for B02 Model:


Output()

ERROR:pymc.stats.convergence:There were 31 divergences after tuning. Increase `target_accept` or reparameterize.


Output()

/usr/local/lib/python3.11/dist-packages/arviz/stats/diagnostics.py:596: RuntimeWarning: invalid value encountered in scalar divide
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
/usr/local/lib/python3.11/dist-packages/arviz/stats/diagnostics.py:991: RuntimeWarning: invalid value encountered in scalar divide
  varsd = varvar / evar / 4
/usr/local/lib/python3.11/dist-packages/arviz/stats/density_utils.py:488: UserWarning: Your data appears to have a single value or no finite values
  warnings.warn("Your data appears to have a single value or no finite values")
/usr/local/lib/python3.11/dist-packages/arviz/plots/plot_utils.py:270: UserWarning: rcParams['plot.max_subplots'] (40) is smaller than the number of variables to plot (500) in plot_posterior, generating only 40 plots
  warnings.warn(
<ipython-input-3-0d9c711024a3>:53: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  plt.tight_layout()


✅ Saved all outputs for model B02 in 'results' folder.


In [ ]:
from google.colab import files
!zip -r results_b02.zip results/
#files.download("results_b02.zip")

	zip warning: name not matched: results/

zip error: Nothing to do! (try: zip -r results_b02.zip . -i results/)


In [ ]:
# Run and save results for B01
print("\nResults for B01 Model:")
idata_b01, waic_b01, loo_b01, bic_b01 = run_and_summarize(test_data, ordinal_predictor_binary_outcome_model, label="b01")


Results for B01 Model:


Output()

ERROR:pymc.stats.convergence:There were 28 divergences after tuning. Increase `target_accept` or reparameterize.


Output()

/usr/local/lib/python3.11/dist-packages/arviz/stats/diagnostics.py:596: RuntimeWarning: invalid value encountered in scalar divide
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
/usr/local/lib/python3.11/dist-packages/arviz/stats/diagnostics.py:991: RuntimeWarning: invalid value encountered in scalar divide
  varsd = varvar / evar / 4
/usr/local/lib/python3.11/dist-packages/arviz/stats/density_utils.py:488: UserWarning: Your data appears to have a single value or no finite values
  warnings.warn("Your data appears to have a single value or no finite values")
/usr/local/lib/python3.11/dist-packages/arviz/plots/plot_utils.py:270: UserWarning: rcParams['plot.max_subplots'] (40) is smaller than the number of variables to plot (500) in plot_posterior, generating only 40 plots
  warnings.warn(
<ipython-input-7-0d9c711024a3>:53: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  plt.tight_layout()


✅ Saved all outputs for model B01 in 'results' folder.


In [ ]:
from google.colab import files
!zip -r results_b01.zip results/
files.download("results_b01.zip")

  adding: results/ (stored 0%)
  adding: results/idata_b02.nc (deflated 22%)
  adding: results/summary_b01.csv (deflated 95%)
  adding: results/model_metrics_b02.md (deflated 77%)
  adding: results/trace_b01.png (deflated 3%)
  adding: results/idata_b01.nc (deflated 22%)
  adding: results/posterior_p_b01.png (deflated 21%)
  adding: results/trace_b02.png (deflated 3%)
  adding: results/model_metrics_b01.md (deflated 77%)
  adding: results/ppc_b02.png (deflated 24%)
  adding: results/posterior_p_b02.png (deflated 21%)
  adding: results/summary_b02.csv (deflated 95%)
  adding: results/ppc_b01.png (deflated 24%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Tesing the model using the survey dataset

In [7]:
# Data Generation
#%run /content/drive/MyDrive/Bayesian\ path\ analysis/code_snippets.py
%run code_snippets.py

25-26/77-78 27-28/79-80 29-30/81-82 31-32/83-84 33-34/85-86 35-36/87-88 37-38/89-90 39-40/91-92 41-42/93-94 43-44/95-96


<Figure size 640x480 with 0 Axes>

In [8]:
# Rename columns to C999 format
new_columns = {old_col: f'C{i:03d}' for i, old_col in enumerate(d.columns)}
d = d.rename(columns=new_columns)

In [9]:
d.columns

Index(['C000', 'C001', 'C002', 'C003', 'C004', 'C005', 'C006', 'C007', 'C008',
       'C009',
       ...
       'C102', 'C103', 'C104', 'C105', 'C106', 'C107', 'C108', 'C109', 'C110',
       'C111'],
      dtype='object', length=112)

In [10]:
#Merge the predictor columns
d["I_I"] = merge_columns(d["C027"], d["C079"])
print("\nDataframe with merged \"I_I\" columns (first 5 rows of relevant columns):")
print(d[["C027", "C079", "I_I"]].head())


Dataframe with merged "I_I" columns (first 5 rows of relevant columns):
              C027             C079              I_I
0              NaN       Egyetértek       Egyetértek
1              NaN       Egyetértek       Egyetértek
2  Nem értek egyet              NaN  Nem értek egyet
3              NaN  Nem értek egyet  Nem értek egyet
4              NaN  Nem értek egyet  Nem értek egyet


In [11]:
#Prepare the outcome variable
print("\nUnique values in the outcome columns (C004):")
print(d["C004"].unique())


Unique values in the outcome columns (C004):
['Nem' 'Igen']


In [12]:
d["outcome_binary"] = d["C004"].apply(lambda x: 1 if x == "Igen" else 0)
print("\nFirst few rows with binary outcome:")
print(d[["C004", "outcome_binary"]].head())


First few rows with binary outcome:
   C004  outcome_binary
0   Nem               0
1   Nem               0
2  Igen               1
3   Nem               0
4   Nem               0


In [13]:
#Prepare the predictor variable (I_I) - Dichotomize into agree/disagree
print("\nUnique values in the merged 'I_I' column:")
print(d["I_I"].unique())


Unique values in the merged 'I_I' column:
['Egyetértek' 'Nem értek egyet' 'Teljesen egyetértek'
 'Egyáltalán nem értek egyet']


In [10]:
d.describe()

,C005,C006,C007,C008,C009,C010,C011,C012,C013,C014,...,C068,C069,C070,C071,C072,C073,C074,C075,C076,outcome_binary
count,125.000000,125.000000,125.000000,125.00000,125.000000,125.000000,125.000000,125.000000,125.000000,125.000000,...,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,333.000000
mean,14.648000,10.200000,16.688000,12.88800,13.352000,14.024000,16.904000,10.472000,12.976000,11.760000,...,7.817308,7.456731,9.658654,8.533654,5.875000,4.687500,8.879808,8.721154,6.408654,0.375375
std,4.882913,5.840625,3.798166,4.94847,4.924029,3.766285,2.613316,4.363662,3.670862,4.135605,...,4.417279,4.014232,4.655662,4.684110,4.111225,3.955481,5.625166,5.349535,6.019399,0.484948
min,2.000000,1.000000,4.000000,1.00000,1.000000,3.000000,9.000000,1.000000,4.000000,1.000000,...,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,11.000000,5.000000,15.000000,10.00000,11.000000,12.000000,15.000000,7.000000,10.000000,9.000000,...,4.000000,5.000000,6.000000,5.000000,3.000000,2.000000,3.000000,4.000000,1.000000,0.000000
50%,16.000000,10.000000,18.000000,14.00000,14.000000,15.000000,17.000000,11.000000,13.000000,11.000000,...,8.000000,7.000000,9.000000,7.000000,5.000000,4.000000,9.000000,9.000000,4.000000,0.000000
75%,19.000000,15.000000,20.000000,17.00000,17.000000,17.000000,19.000000,13.000000,16.000000,14.000000,...,11.000000,10.000000,13.250000,12.000000,8.000000,7.000000,14.000000,13.000000,11.000000,1.000000
max,20.000000,20.000000,20.000000,20.00000,20.000000,20.000000,20.000000,19.000000,20.000000,20.000000,...,19.000000,19.000000,20.000000,19.000000,17.000000,20.000000,20.000000,20.000000,20.000000,1.000000


In [11]:
d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 333 entries, 0 to 332
Columns: 114 entries, C000 to outcome_binary
dtypes: float64(40), int64(1), object(73)
memory usage: 296.7+ KB


In [12]:
d.isna().sum()

,0
C000,0
C001,0
C002,0
C003,0
C004,0
...,...
C109,24
C110,25
C111,0
I_I,0


In [14]:
likert_mapping = {
    "Egyáltalán nem értek egyet": 1,
    "Nem értek egyet": 2,
    "Egyetértek": 3,
    "Teljesen egyetértek": 4
}
d["I_I_numeric"] = d["I_I"].map(likert_mapping)

In [15]:
d.describe()

,C005,C006,C007,C008,C009,C010,C011,C012,C013,C014,...,C069,C070,C071,C072,C073,C074,C075,C076,outcome_binary,I_I_numeric
count,125.000000,125.000000,125.000000,125.00000,125.000000,125.000000,125.000000,125.000000,125.000000,125.000000,...,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,333.000000,333.000000
mean,14.648000,10.200000,16.688000,12.88800,13.352000,14.024000,16.904000,10.472000,12.976000,11.760000,...,7.456731,9.658654,8.533654,5.875000,4.687500,8.879808,8.721154,6.408654,0.375375,2.729730
std,4.882913,5.840625,3.798166,4.94847,4.924029,3.766285,2.613316,4.363662,3.670862,4.135605,...,4.014232,4.655662,4.684110,4.111225,3.955481,5.625166,5.349535,6.019399,0.484948,0.842536
min,2.000000,1.000000,4.000000,1.00000,1.000000,3.000000,9.000000,1.000000,4.000000,1.000000,...,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000
25%,11.000000,5.000000,15.000000,10.00000,11.000000,12.000000,15.000000,7.000000,10.000000,9.000000,...,5.000000,6.000000,5.000000,3.000000,2.000000,3.000000,4.000000,1.000000,0.000000,2.000000
50%,16.000000,10.000000,18.000000,14.00000,14.000000,15.000000,17.000000,11.000000,13.000000,11.000000,...,7.000000,9.000000,7.000000,5.000000,4.000000,9.000000,9.000000,4.000000,0.000000,3.000000
75%,19.000000,15.000000,20.000000,17.00000,17.000000,17.000000,19.000000,13.000000,16.000000,14.000000,...,10.000000,13.250000,12.000000,8.000000,7.000000,14.000000,13.000000,11.000000,1.000000,3.000000
max,20.000000,20.000000,20.000000,20.00000,20.000000,20.000000,20.000000,19.000000,20.000000,20.000000,...,19.000000,20.000000,19.000000,17.000000,20.000000,20.000000,20.000000,20.000000,1.000000,4.000000


In [15]:
#Prepare data for the Bayesian model
y = d["outcome_binary"].values
X = d["I_I_numeric"].values

In [16]:
d_model = pd.DataFrame({
        "predictor": X,
        "outcome": y
    })
d_model.head()

,predictor,outcome
0,3,0
1,3,0
2,2,1
3,2,0
4,2,0


In [17]:
# Run and save results for B02 for survey data
print("\nResults for B02 Model on survey data:")
idata_b02_s, waic_b02_s, loo_b02_s, bic_b02_s = run_and_summarize(d_model, ordinal_predictor_binary_outcome_model, label="b02")


Results for B02 Model on survey data:


Output()

ERROR:pymc.stats.convergence:There were 10 divergences after tuning. Increase `target_accept` or reparameterize.


Output()

/usr/local/lib/python3.11/dist-packages/arviz/stats/diagnostics.py:596: RuntimeWarning: invalid value encountered in scalar divide
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
/usr/local/lib/python3.11/dist-packages/arviz/stats/diagnostics.py:991: RuntimeWarning: invalid value encountered in scalar divide
  varsd = varvar / evar / 4
/usr/local/lib/python3.11/dist-packages/arviz/stats/density_utils.py:488: UserWarning: Your data appears to have a single value or no finite values
  warnings.warn("Your data appears to have a single value or no finite values")
/usr/local/lib/python3.11/dist-packages/arviz/plots/plot_utils.py:270: UserWarning: rcParams['plot.max_subplots'] (40) is smaller than the number of variables to plot (333) in plot_posterior, generating only 40 plots
  warnings.warn(


✅ Saved all outputs for model B02 in 'results' folder.


In [18]:
from google.colab import files
!zip -r survey_results_b02.zip results/
files.download("survey_results_b02.zip")

  adding: results/ (stored 0%)
  adding: results/posterior_p_b02.png (deflated 19%)
  adding: results/summary_b02.csv (deflated 93%)
  adding: results/trace_b02.png (deflated 4%)
  adding: results/idata_b02.nc (deflated 23%)
  adding: results/model_metrics_b02.md (deflated 77%)
  adding: results/ppc_b02.png (deflated 24%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>